In [1]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import date

In [2]:
df_old = pd.read_csv("brd_covid_19.csv",parse_dates=[7])

In [3]:
df_old = df_old.sort_values(["bundesland", "datum"])
df_old.drop_duplicates(inplace=True)
df_old = df_old.reset_index().drop("index",axis = 1)

In [4]:
today = date.today()

In [5]:
URL = "https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html"

In [6]:
page = requests.get(URL)

In [7]:
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
results = soup.find_all("tr")

In [9]:
bundesland = [re.sub('<[^<]+?>', '', str(elm.find_all("td")[0])).replace("\xad","") for idx, elm in enumerate(results) if idx > 1]
faelle = [re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])).replace(".","") for idx, elm in enumerate(results) if idx > 1]
neue_faelle = [re.sub('<[^<]+?>', '', str(elm.find_all("td")[2])).replace(".","") for idx, elm in enumerate(results) if idx > 1]
faelle_per_capita = [re.sub('<[^<]+?>', '', str(elm.find_all("td")[3])).replace(".","") for idx, elm in enumerate(results) if idx > 1]
tod = [re.sub('<[^<]+?>', '', str(elm.find_all("td")[4])).replace(".","") for idx, elm in enumerate(results) if idx > 1]

In [10]:
lat={"Baden-Württemberg":48.5, "Bayern":48.9, "Berlin":52.5, "Brandenburg":52, "Bremen":53.1, "Hamburg":53.5, 
     "Hessen":50.7, "Mecklenburg-Vorpommern":53.8, "Niedersachsen":52.57, "Nordrhein-Westfalen":51.6,"Rheinland-Pfalz":50, 
     "Saarland":49.4, "Sachsen":51.05, "Sachsen-Anhalt":52, "Schleswig-Holstein":54.2, "Thüringen":50.85, "Gesamt":51.06}
long={"Baden-Württemberg":9, "Bayern":11.4, "Berlin":13.4, "Brandenburg":13.87, "Bremen":8.8, "Hamburg":10, 
      "Hessen":9, "Mecklenburg-Vorpommern":12.5,"Niedersachsen":10, "Nordrhein-Westfalen":7.5,"Rheinland-Pfalz":7.3,
      "Saarland":6.97,"Sachsen":13.5, "Sachsen-Anhalt":11.65,"Schleswig-Holstein":9.9, "Thüringen":11.13,"Gesamt":10}


In [11]:
df=pd.DataFrame(data=[bundesland,faelle,tod,neue_faelle,faelle_per_capita]).T
    

In [12]:
df["lat"]="NaN"
df["long"]="NaN"
df["date"] = today

In [13]:
df.columns = ["bundesland","faelle","tod","neue_faelle","faelle_per_capita","lat","long","datum"]

In [14]:
df.datum = pd.to_datetime(df.datum)

In [15]:
df.bundesland.replace("Schleswig Holstein","Schleswig-Holstein", inplace=True)
df.tod = df.tod.replace(to_replace = "", value = "0", regex = True)

In [16]:
df["lat"]=[lat[elm] for elm in df.bundesland.values]
df["long"]=[long[elm] for elm in df.bundesland.values]


In [17]:
#df.lat = df.lat.astype("float")
#df.long = df.long.astype("float")

df.tod = df.tod.astype("int64")
df.faelle = df.faelle.astype("int64")
df.neue_faelle = df.neue_faelle.astype("int64")
df.faelle_per_capita = df.faelle_per_capita.astype("float64")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   bundesland         17 non-null     object        
 1   faelle             17 non-null     int64         
 2   tod                17 non-null     int64         
 3   neue_faelle        17 non-null     int64         
 4   faelle_per_capita  17 non-null     float64       
 5   lat                17 non-null     float64       
 6   long               17 non-null     float64       
 7   datum              17 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(1)
memory usage: 1.2+ KB


In [19]:
df

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
0,Baden-Württemberg,3807,21,139,34.0,48.50,9.00,2020-03-22
1,Bayern,3650,21,690,28.0,48.90,11.40,2020-03-22
2,Berlin,1024,1,158,27.0,52.50,13.40,2020-03-22
3,Brandenburg,274,0,20,11.0,52.00,13.87,2020-03-22
4,Bremen,165,0,23,24.0,53.10,8.80,2020-03-22
5,Hamburg,872,0,285,47.0,53.50,10.00,2020-03-22
6,Hessen,1175,2,95,19.0,50.70,9.00,2020-03-22
7,Mecklenburg-Vorpommern,172,0,7,11.0,53.80,12.50,2020-03-22
8,Niedersachsen,1306,1,283,16.0,52.57,10.00,2020-03-22
9,Nordrhein-Westfalen,3545,6,3,20.0,51.60,7.50,2020-03-22


In [21]:
pd.concat([df_old,df]).reset_index().drop("index",axis=1).tail(30)

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
351,Thüringen,4,0,2,0.20,50.85,11.13,2020-03-10
352,Thüringen,10,0,6,0.50,50.85,11.13,2020-03-11
353,Thüringen,14,0,4,0.70,50.85,11.13,2020-03-12
354,Thüringen,29,0,15,1.40,50.85,11.13,2020-03-13
355,Thüringen,46,0,17,2.10,50.85,11.13,2020-03-14
356,Thüringen,51,0,5,2.40,50.85,11.13,2020-03-15
357,Thüringen,55,0,4,2.60,50.85,11.13,2020-03-16
358,Thüringen,51,0,15,2.40,50.85,11.13,2020-03-17
359,Thüringen,74,0,23,3.50,50.85,11.13,2020-03-18
360,Thüringen,98,0,24,4.57,50.85,11.13,2020-03-19


In [72]:
df=pd.concat([df_old,df]).reset_index().drop("index",axis=1)

In [73]:
df[df["bundesland"]=="Bayern"]

,bundesland,faelle,tod,neue_faelle,faelle_per_capita,lat,long,datum
23,Bayern,15,0,0,0.10,48.9,11.4,2020-02-28
24,Bayern,15,0,0,0.10,48.9,11.4,2020-02-29
25,Bayern,19,0,4,0.10,48.9,11.4,2020-03-01
26,Bayern,25,0,6,0.20,48.9,11.4,2020-03-02
27,Bayern,35,0,10,0.30,48.9,11.4,2020-03-03
28,Bayern,48,0,13,0.40,48.9,11.4,2020-03-04
29,Bayern,52,0,4,0.40,48.9,11.4,2020-03-05
30,Bayern,79,0,27,0.60,48.9,11.4,2020-03-06
31,Bayern,117,0,38,0.90,48.9,11.4,2020-03-07
32,Bayern,148,0,31,1.10,48.9,11.4,2020-03-08


In [74]:
df_old.to_csv ('old_data/brd_covid_19'+str(today)+".csv", index = False, header=True)

In [75]:
df.to_csv ('brd_covid_19.csv', index = False, header=True)